In [11]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.nn import Module, Sequential
from torch.utils.data import DataLoader, Subset
from torch import optim
from tqdm import tqdm
import pandas as pd
import wandb

### Descărcare și normalizare date

In [23]:
image_to_tensor = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,)),
    transforms.Lambda(torch.flatten)
])
train_set = torchvision.datasets.MNIST(
    root='./training',
    train=True,
    download=True,
    transform=image_to_tensor,
)

test_set = torchvision.datasets.MNIST(
    root='./test',
    train=False,
    download=True,
    transform=image_to_tensor
)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### Definirea modelului

In [24]:
class MyNN(Module):
    def __init__(self, input_size, num_classes, dropout_rate=0.2):
        super(MyNN, self).__init__()
        self.model = Sequential(
            torch.nn.Linear(input_size, 256),
            torch.nn.LeakyReLU(),
            torch.nn.AlphaDropout(dropout_rate, inplace=True),
            torch.nn.Linear(256, 100),
            torch.nn.LeakyReLU(),
            torch.nn.AlphaDropout(dropout_rate, inplace=True),
            torch.nn.Linear(100, num_classes)
        )
    def forward(self, x):
        return self.model(x)

### Dataloaders, optimizatori, hiperparametrii

In [29]:
# OPTIMAL HYPERPARAMETERS
INPUT_SIZE = 784
OUTPUT_CLASSES = 10
LEARNING_RATE = 0.0015
BATCH_SIZE = 256
LR_DECAY_RATE = 0.2
DROPOUT_RATE = 0.14

EPOCHS = 100

criterion = torch.nn.CrossEntropyLoss()

In [15]:
def check_accuracy(loader, model: Module):
    num_correct = 0
    num_samples = 0
    model.eval()
    
    with torch.no_grad():
        for image, target in loader:
            image, target = image.to(device), target.to(device)
            output = model(image)
            pred = output.argmax(dim=1, keepdim=True)
            num_correct += pred.eq(target.view_as(pred)).sum().item()
            num_samples += image.size()[0]
            
    model.train()
    return num_correct / num_samples

In [16]:
def generate_submission(test_set, model: Module):
    model.eval()
    
    with torch.no_grad():
        for i, (image, _) in enumerate(test_set):
            image = image.to(device)
            output = model(image)
            pred = output.argmax(dim=0)
            submission_data["ID"].append(i)
            submission_data["target"].append(pred.item())
    data_frame = pd.DataFrame(submission_data)
    data_frame.to_csv('submission.csv', index=False)
    model.train()

### Configurarea w&b pentru a găsi cea mai bună configurație

In [17]:
wandb.login()
sweep_config = {
    'method': 'random',
    'metric': {
        'name': 'accuracy',
        'goal': 'maximize'
    },
    'parameters': {
        'optimizer': {
            'value': 'adam'
        },
        'learning_rate': {
            'distribution': 'uniform',
            'min': 0.0001,
            'max': 0.01
        },
        'batch_size': {
            'values': [128, 256, 512]
        },
        'dropout_rate': {
            'distribution': 'uniform',
            'min': 0.1,
            'max': 0.5
        },
        'lr_decay_rate': {
            'distribution': 'uniform',
            'min': 0.1,
            'max': 0.5
        },
        'epochs': {
            'value': 25
        }
    }
}

### Antrenarea și uploadarea modelului

In [26]:
def gen_model(learning_rate, optimizer_name, dropout_rate, lr_decay_rate) -> tuple[Module, optim.Optimizer, optim.lr_scheduler.StepLR]:
    model = MyNN(INPUT_SIZE, OUTPUT_CLASSES, dropout_rate).to(device)
    if optimizer_name == 'adam':
        optimizer = optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=0.0001)
    else:
        optimizer = optim.SGD(model.parameters(), lr=learning_rate,momentum=0.9)
    rate_reducer = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=lr_decay_rate)
    return model, optimizer, rate_reducer

def gen_loader(batch_size) -> tuple[DataLoader, DataLoader]:
    train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, drop_last=True, num_workers=4)
    test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=True, drop_last=True, num_workers=4)
    return train_loader, test_loader

def train_epoch(loader, model: Module, optimizer):
    model.train()
    for _, (images, labels) in enumerate(tqdm(loader)):
        images = images.to(device)
        labels = labels.to(device)
        
        scores = model(images)
        loss = criterion(scores, labels)
        
        optimizer.zero_grad()
        loss.backward()
        
        optimizer.step()

In [19]:
def train(config=None):
    with wandb.init(config=config):
        config = wandb.config
        
        model, optimizer, rate_reducer = gen_model(config.learning_rate, config.optimizer, config.dropout_rate, config.lr_decay_rate)
        train_loader, test_loader = gen_loader(config.batch_size)
        
        for epoch in range(config.epochs):
            train_epoch(train_loader, model, optimizer)
            accuracy = check_accuracy(test_loader, model)
            wandb.log({'accuracy': accuracy, 'epoch': epoch})
        

In [20]:
sweep_id = wandb.sweep(sweep_config, project='assignment-4')
wandb.agent(sweep_id, function=train, count=32)

Create sweep with ID: 2hr7fem4
Sweep URL: https://wandb.ai/ionut-calin-uaic/assignment-4/sweeps/2hr7fem4


wandb: Agent Starting Run: gr0f6g3p with config:
wandb: 	batch_size: 128
wandb: 	dropout_rate: 0.2234705600614241
wandb: 	epochs: 25
wandb: 	learning_rate: 0.006419541136053063
wandb: 	lr_decay_rate: 0.18230544662975137
wandb: 	optimizer: adam


100%|██████████| 468/468 [00:07<00:00, 63.17it/s] 


accuracy,▂▁▄▄▅▅▅▆▆▆▇▆▆▆▇▇▆▇▇▃▇▆█▇█
epoch,▁▁▂▂▂▂▃▃▃▄▄▄▅▅▅▅▆▆▆▇▇▇▇██
accuracy,0.97246
epoch,24


wandb: Agent Starting Run: khw8ar4t with config:
wandb: 	batch_size: 128
wandb: 	dropout_rate: 0.47166891949594303
wandb: 	epochs: 25
wandb: 	learning_rate: 0.003448618994183813
wandb: 	lr_decay_rate: 0.3200344549602876
wandb: 	optimizer: adam


...

100%|██████████| 234/234 [00:07<00:00, 32.52it/s]


accuracy,▁▄▄▆▆▆▆▇▇▇▇▇▇████████████
epoch,▁▁▂▂▂▂▃▃▃▄▄▄▅▅▅▅▆▆▆▇▇▇▇██
accuracy,0.98097
epoch,24


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: 39pz3gdy with config:
wandb: 	batch_size: 128
wandb: 	dropout_rate: 0.25252406738978295
wandb: 	epochs: 25
wandb: 	learning_rate: 0.009508800805859596
wandb: 	lr_decay_rate: 0.16256932002576924
wandb: 	optimizer: adam


100%|██████████| 468/468 [00:07<00:00, 62.59it/s] 


accuracy,▂▁▅▆▆▅▆▆▇█▆▇██▇▇▇█▆▇▇▇▆██
epoch,▁▁▂▂▂▂▃▃▃▄▄▄▅▅▅▅▆▆▆▇▇▇▇██
accuracy,0.96524
epoch,24


### Rularea cu hiperparametrii relativ optimi

In [30]:
submission_data = {
    "ID": [],
    "target": [],
}

train_loader, test_loader = gen_loader(BATCH_SIZE)
model, optimizer, rate_reducer = gen_model(LEARNING_RATE, 'adam', DROPOUT_RATE, LR_DECAY_RATE)

max_accuracy = 0.0

for epoch in range(EPOCHS):
    train_epoch(train_loader, model, optimizer)
    test_accuracy = check_accuracy(test_loader, model)
    train_accuracy = check_accuracy(train_loader, model)
    if test_accuracy > max_accuracy:
        max_accuracy = test_accuracy
        generate_submission(test_set, model)
    print(f'Epoch {epoch}\n\ttrain accuracy: {train_accuracy*100:.2f}\n\tvalidation accuracy: {test_accuracy*100:.2f}')
    
print(f'Max accuracy on validation: {max_accuracy*100:.2f}')

100%|██████████| 234/234 [00:08<00:00, 27.41it/s]


Epoch 0
	train accuracy: 91.75
	validation accuracy: 92.26


100%|██████████| 234/234 [00:08<00:00, 28.14it/s]


Epoch 1
	train accuracy: 94.80
	validation accuracy: 94.89


100%|██████████| 234/234 [00:08<00:00, 28.50it/s]


...

Epoch 97
	train accuracy: 99.89
	validation accuracy: 98.34


100%|██████████| 234/234 [00:08<00:00, 29.03it/s]


Epoch 98
	train accuracy: 99.94
	validation accuracy: 98.43


100%|██████████| 234/234 [00:08<00:00, 28.41it/s]


Epoch 99
	train accuracy: 99.90
	validation accuracy: 98.45
Max accuracy on validation: 98.57
